In [1]:
import pandas as pd
from dhlab_utils import * 

In [2]:
fg_user_known = pd.read_csv('../datas/interim/fg_user_known.csv')
fg_user_unknown = pd.read_csv('../datas/interim/fg_user_unknown.csv')

In [10]:
fg_user_known.sample(5)

,Unnamed: 0,before,after,previous_output,type_of_change,category,significance,finegrain_output_raw,after_json_text,username,date,article
7,28,#REDIRECT: [[Crimea]],"--- \n+++ \n@@ -98,8 +98,8 @@\n \n ===2014 Cri...","The change made is the addition of the term ""2...",addition of terms and rephrasing,Framing & Emphasis Shifts,The addition of '2014 Ukrainian revolution' sh...,"```json\n{\n ""detected_changes"": [\n {\n ...","--- \n+++ \n@@ -98,8 +98,8 @@\n \n ===2014 Cri...",Spiritofstgeorge,unknown,unknown
200,725,[[File:1 марта 2014 года в Донецке. У областно...,"--- \n+++ \n@@ -426,14 +426,15 @@\n * '''{{fla...",The change made is the addition of text regard...,addition of text and citations,Selective Insertion,The addition of details about the Kharkov Peop...,"```json\n{\n ""detected_changes"": [\n {\n ...","--- \n+++ \n@@ -426,14 +426,15 @@\n * '''{{fla...",180.210.208.1,unknown,unknown
183,662,The '''Eastern Front''' refers to a theatre of...,"--- \n+++ \n@@ -271,7 +271,7 @@\n \n In the en...",The change made in this revision is the **refo...,reformatting and punctuation adjustment,Selective Insertion,The edit emphasizes the strength and capabilit...,"```json\n{\n ""detected_changes"": [\n {\n ...","--- \n+++ \n@@ -271,7 +271,7 @@\n \n In the en...",WikiCleanerBot,unknown,unknown
84,306,The war between Germany and the Soviet Union d...,"--- \n+++ \n@@ -20,7 +20,7 @@\n |[[East Prussi...","Changed ""Fall of [[Nazi Germany]]"" to ""Fall of...",Terminology Manipulation,1. Terminology Manipulation,The change from 'Nazi Germany' to 'Third Reich...,"```json\n{\n ""detected_changes"": [\n {\n ...","--- \n+++ \n@@ -20,7 +20,7 @@\n |[[East Prussi...",Denniss,unknown,unknown
163,586,#REDIRECT: [[Crimea]],"--- \n+++ \n@@ -405,7 +405,7 @@\n | 1,180,441\...","Changed the demographic figures to include a ""...",Insertion of a dubious tag,Framing & Emphasis Shifts,The addition of the 'dubious' tag to demograph...,"```json\n{\n ""detected_changes"": [{""before"": ...","--- \n+++ \n@@ -405,7 +405,7 @@\n | 1,180,441\...",Toddy1,unknown,unknown


In [8]:
fg_user_unknown.sample()

,Unnamed: 0.1,Unnamed: 0,before,after,previous_output,type_of_change,category,significance,finegrain_output_raw,after_json_text,username,date,article
854,1139,1139,Capital and largest city of the [[Ukraine]] (2...,"--- \n+++ \n@@ -61,7 +61,7 @@\n |blank_info ...",The change made is a rephrasing of the introdu...,rephrasing and contextualization,Terminology Manipulation,This change emphasizes the Ukrainian name 'Kyi...,"```json\n{\n ""detected_changes"": [\n {\n ...","--- \n+++ \n@@ -61,7 +61,7 @@\n |blank_info ...",unknown,unknown,unknown
